## customer lifetime value (CLV)
### The customer lifetime value is a measurement of how much a company expects to earn from an average customer in a lifetime. It can be historical - where we sum up each customer's profit and get the actual customer value. Another alternative is to predict the customer lifetime value which is a bit more complex

In [71]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [6]:
retail = pd.read_excel(r"C:\Users\ASUS\Desktop\github\online_retail_II.xlsx")

In [7]:
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [8]:
retail.shape

(525461, 8)

In [9]:
print("number of unique customers:", retail['Customer ID'].nunique())

number of unique customers: 4383


In [10]:
#checking duplicates
print(f"there are {retail.duplicated().sum()} duplicated rows")

there are 6865 duplicated rows


In [12]:
retail = retail.drop_duplicates(keep='first')

In [13]:
#checking missing values
retail.isna().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107833
Country             0
dtype: int64

In [14]:
retail = retail[retail['Customer ID'].notna()]

In [15]:
# checking types of columns
retail.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

## Calculating Cohort Index

In [16]:
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'])

In [17]:
retail['InvoiceDate']

0        2009-12-01 07:45:00
1        2009-12-01 07:45:00
2        2009-12-01 07:45:00
3        2009-12-01 07:45:00
4        2009-12-01 07:45:00
                 ...        
525456   2010-12-09 20:01:00
525457   2010-12-09 20:01:00
525458   2010-12-09 20:01:00
525459   2010-12-09 20:01:00
525460   2010-12-09 20:01:00
Name: InvoiceDate, Length: 410763, dtype: datetime64[ns]

In [18]:
retail['YearMonth'] = retail['InvoiceDate'].dt.strftime("%Y-%m")

In [19]:
retail['YearMonth'] 

0         2009-12
1         2009-12
2         2009-12
3         2009-12
4         2009-12
           ...   
525456    2010-12
525457    2010-12
525458    2010-12
525459    2010-12
525460    2010-12
Name: YearMonth, Length: 410763, dtype: object

In [20]:
retail['YearMonth'] = pd.to_datetime(retail['YearMonth'])

In [21]:
retail['YearMonth']

0        2009-12-01
1        2009-12-01
2        2009-12-01
3        2009-12-01
4        2009-12-01
            ...    
525456   2010-12-01
525457   2010-12-01
525458   2010-12-01
525459   2010-12-01
525460   2010-12-01
Name: YearMonth, Length: 410763, dtype: datetime64[ns]

In [22]:
retail['CohortMonth'] = retail.groupby(by=['Customer ID'])['YearMonth'].transform('min')

In [24]:
def calculate_cohort_index(df, transactions_col, cohort_month):
    
    year_diff = df[transactions_col].dt.year - df[cohort_month].dt.year

    month_diff = df[transactions_col].dt.month - df[cohort_month].dt.month
    
    cohort_index = year_diff * 12 + month_diff + 1
    
    return cohort_index

In [25]:
retail['CohortIndex'] = calculate_cohort_index(retail, 'InvoiceDate', 'CohortMonth')

In [45]:
retail.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,YearMonth,CohortMonth,CohortIndex,PurchaseSize
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom,2010-12-01,2009-12-01,13,5.90
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,2010-12-01,2009-12-01,13,3.75
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,2010-12-01,2009-12-01,13,3.75
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,2010-12-01,2009-12-01,13,7.50
525460,538171,21931,JUMBO STORAGE BAG SUKI,2,2010-12-09 20:01:00,1.95,17530.0,United Kingdom,2010-12-01,2009-12-01,13,3.90


## Calculating Average Active Users per Cohort

In [27]:
users_in_cohorts = retail.groupby(by=['CohortMonth', 'CohortIndex'])['Customer ID'].nunique()


In [28]:
users_in_cohorts = users_in_cohorts.reset_index()
users_in_cohorts.head()

,CohortMonth,CohortIndex,Customer ID
0,2009-12-01,1,1045
1,2009-12-01,2,392
2,2009-12-01,3,358
3,2009-12-01,4,447
4,2009-12-01,5,410


In [29]:
cohorts_fin = users_in_cohorts.pivot(index='CohortMonth', columns='CohortIndex', values='Customer ID')
cohorts_fin

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2009-12-01,1045.0,392.0,358.0,447.0,410.0,408.0,408.0,374.0,355.0,392.0,452.0,518.0,260.0
2010-01-01,394.0,86.0,119.0,120.0,110.0,115.0,105.0,91.0,114.0,134.0,122.0,37.0,NaN
2010-02-01,363.0,109.0,82.0,110.0,93.0,76.0,79.0,103.0,100.0,106.0,32.0,NaN,NaN
2010-03-01,436.0,95.0,113.0,103.0,100.0,87.0,105.0,130.0,126.0,36.0,NaN,NaN,NaN
2010-04-01,291.0,67.0,58.0,47.0,54.0,67.0,79.0,76.0,22.0,NaN,NaN,NaN,NaN
2010-05-01,254.0,49.0,45.0,49.0,48.0,66.0,56.0,22.0,NaN,NaN,NaN,NaN,NaN
2010-06-01,269.0,58.0,53.0,55.0,62.0,76.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,183.0,38.0,37.0,52.0,55.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,158.0,39.0,50.0,51.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Calculating Retention Rates

In [30]:
cohort_sizes = cohorts_fin.iloc[:, 0]
retention = cohorts_fin.divide(cohort_sizes, axis='rows').round(3)

In [47]:
cohorts_fin

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2009-12-01,1045.0,392.0,358.0,447.0,410.0,408.0,408.0,374.0,355.0,392.0,452.0,518.0,260.0
2010-01-01,394.0,86.0,119.0,120.0,110.0,115.0,105.0,91.0,114.0,134.0,122.0,37.0,NaN
2010-02-01,363.0,109.0,82.0,110.0,93.0,76.0,79.0,103.0,100.0,106.0,32.0,NaN,NaN
2010-03-01,436.0,95.0,113.0,103.0,100.0,87.0,105.0,130.0,126.0,36.0,NaN,NaN,NaN
2010-04-01,291.0,67.0,58.0,47.0,54.0,67.0,79.0,76.0,22.0,NaN,NaN,NaN,NaN
2010-05-01,254.0,49.0,45.0,49.0,48.0,66.0,56.0,22.0,NaN,NaN,NaN,NaN,NaN
2010-06-01,269.0,58.0,53.0,55.0,62.0,76.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,183.0,38.0,37.0,52.0,55.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,158.0,39.0,50.0,51.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
cohort_sizes

CohortMonth
2009-12-01    1045.0
2010-01-01     394.0
2010-02-01     363.0
2010-03-01     436.0
2010-04-01     291.0
2010-05-01     254.0
2010-06-01     269.0
2010-07-01     183.0
2010-08-01     158.0
2010-09-01     242.0
2010-10-01     379.0
2010-11-01     322.0
2010-12-01      47.0
Name: 1, dtype: float64

In [49]:
retention

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2009-12-01,1.0,0.375,0.343,0.428,0.392,0.390,0.390,0.358,0.340,0.375,0.433,0.496,0.249
2010-01-01,1.0,0.218,0.302,0.305,0.279,0.292,0.266,0.231,0.289,0.340,0.310,0.094,NaN
2010-02-01,1.0,0.300,0.226,0.303,0.256,0.209,0.218,0.284,0.275,0.292,0.088,NaN,NaN
2010-03-01,1.0,0.218,0.259,0.236,0.229,0.200,0.241,0.298,0.289,0.083,NaN,NaN,NaN
2010-04-01,1.0,0.230,0.199,0.162,0.186,0.230,0.271,0.261,0.076,NaN,NaN,NaN,NaN
2010-05-01,1.0,0.193,0.177,0.193,0.189,0.260,0.220,0.087,NaN,NaN,NaN,NaN,NaN
2010-06-01,1.0,0.216,0.197,0.204,0.230,0.283,0.074,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,1.0,0.208,0.202,0.284,0.301,0.115,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,1.0,0.247,0.316,0.323,0.127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
cohort_sizes = cohorts_fin.iloc[:, 0]
cohort_sizes

CohortMonth
2009-12-01    1045.0
2010-01-01     394.0
2010-02-01     363.0
2010-03-01     436.0
2010-04-01     291.0
2010-05-01     254.0
2010-06-01     269.0
2010-07-01     183.0
2010-08-01     158.0
2010-09-01     242.0
2010-10-01     379.0
2010-11-01     322.0
2010-12-01      47.0
Name: 1, dtype: float64

## Calculate Basic CLV (revenue-based)
### The most basic lifetime value calculation is to take the average revenue per customer within a certain period, let's say a month, multiply it by the profit margin, and then multiply the result by the average or expected customer lifespan. 

In [31]:
retail['PurchaseSize'] = retail['Quantity']*retail['Price']


In [32]:
# calculate monthly spend per customer
monthly_revenue = retail.groupby(by=['Customer ID', 'YearMonth'])['PurchaseSize'].sum().mean()
monthly_revenue

596.4745756097561

In [33]:
lifespan_months = 36
CLV_basic = monthly_revenue * lifespan_months
CLV_basic

21473.08472195122

## Calculate Granular CLV (revenue-based)
### A more granular version of the basic customer lifetime value formula looks at each transaction. Here, we multiply the average revenue per purchase or transaction with the average frequency within a defined period - for example a month - and then multiply that with the profit margin. Afterwards, we multiply the result with the average customer lifespan. This method accounts for the frequency of the transactions within a certain timeframe, as well as the average revenue per transaction, therefore capturing more granular data points. Still, it does not account for customer retention rates, and assumes the frequency and revenue per transaction will stay the same within the defined lifespan.

In [34]:
# first mean() averages the revenue per invoice 
# second mean() averages the revenue per purchase
revenue_per_purchase = retail.groupby(by=['Invoice'])['PurchaseSize'].mean().mean()
revenue_per_purchase

28.78630387308043

In [35]:
# calculate average number of unique invoices per customer each month
avg_frequency_per_month = retail.groupby(by=['Customer ID', 'YearMonth'])['Invoice'].nunique().mean()
avg_frequency_per_month

1.6920373027259685

In [36]:
CLV_granular = revenue_per_purchase * avg_frequency_per_month * lifespan_months
CLV_granular

1753.469998590856

## Calculate Traditional CLV (revenue-based)
### The traditional formula is the most popular descriptive customer lifetime value technique. It incorporates retention and churn rates. We calculate it by multiplying the average revenue with the profit margin, and then with the ratio of retention to churn. Churn is defined as 1 minus retention. The retention to churn ratio gives us a multiplier, that acts as a proxy to expected length of the customer lifespan with the company.

In [ ]:
# calculate monthly spend per customer
monthly_revenue = retail.groupby(by=['Customer ID', 'YearMonth'])['PurchaseSize'].sum().mean()
monthly_revenue

In [37]:
#calculate average retention rate per customer
avg_retention = retention.iloc[:, 1:].mean().mean()
avg_retention

0.2553991372053872

In [38]:
avg_churn = 1 - avg_retention
avg_churn

0.7446008627946128

In [39]:
CLV_traditional = monthly_revenue * (avg_retention/avg_churn)
CLV_traditional

204.5916135578018

In [40]:
print("Average CLV: {:.2f} USD at {:.1%} average retention rate and {:.2f} USD average monthly revenue".format(CLV_traditional, avg_retention, monthly_revenue))


Average CLV: 204.59 USD at 25.5% average retention rate and 596.47 USD average monthly revenue


## Data preparation for purchase prediction
### We will predict the number of purchases for the future month. To do that, we need to decide how we will define that future month. We can print out the number of observations we have in each invoice month, and we can see that there are enough observations in all of them.

In [46]:
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,YearMonth,CohortMonth,CohortIndex,PurchaseSize
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009-12-01,2009-12-01,1,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01,2009-12-01,1,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009-12-01,2009-12-01,1,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009-12-01,2009-12-01,1,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009-12-01,2009-12-01,1,30.0


## RFM

In [50]:
retail.groupby(["YearMonth"]).size()

YearMonth
2009-12-01    31276
2010-01-01    22122
2010-02-01    23578
2010-03-01    32592
2010-04-01    27429
2010-05-01    29191
2010-06-01    31447
2010-07-01    27357
2010-08-01    26577
2010-09-01    34911
2010-10-01    49719
2010-11-01    60081
2010-12-01    14483
dtype: int64

In [55]:
now= dt.datetime(2010,12,1)

In [58]:
# Calculate recency by subtracting current date from the latest InvoiceDate
features = retail.groupby('Customer ID').agg({
  'InvoiceDate': lambda x: (now - x.max()).days,
  # Calculate frequency by counting unique number of invoices
  'Invoice': pd.Series.nunique,
  # Calculate monetary value by summing all spend values
  'PurchaseSize': np.sum,
  # Calculate average and total quantity
  'Quantity': ['mean', 'sum']}).reset_index()

# Rename the columns
features.columns = ['CustomerID', 'recency', 'frequency', 'monetary', 'quantity_avg', 'quantity_total']

In [59]:
features

,CustomerID,recency,frequency,monetary,quantity_avg,quantity_total
0,12346.0,57,15,-51.74,1.177778,53
1,12347.0,-7,2,1323.32,11.661972,828
2,12348.0,64,1,222.16,18.650000,373
3,12349.0,33,4,2646.99,9.233645,988
4,12351.0,1,1,300.93,12.428571,261
...,...,...,...,...,...,...
4378,18283.0,8,6,619.37,1.483871,322
4379,18284.0,55,2,436.68,17.000000,493
4380,18285.0,286,1,427.00,12.083333,145
4381,18286.0,102,3,1188.43,8.457143,592


In [62]:
# Build a pivot table counting invoices for each customer monthly
cust_month_tx = pd.pivot_table(data=retail, values='Invoice',
                               index=['Customer ID'], columns=['YearMonth'],
                               aggfunc=pd.Series.nunique, fill_value=0)

# # Store November 2011 sales data column name as a list
# target = ['2011-11']

# # Store target value as `Y`
# Y = cust_month_tx[target]

In [63]:
cust_month_tx

YearMonth,2009-12-01,2010-01-01,2010-02-01,2010-03-01,2010-04-01,2010-05-01,2010-06-01,2010-07-01,2010-08-01,2010-09-01,2010-10-01,2010-11-01,2010-12-01
Customer ID,,,,,,,,,,,,,
12346.0,5,5,0,1,0,0,2,0,0,0,2,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,1,0,1
12348.0,0,0,0,0,0,0,0,0,0,1,0,0,0
12349.0,1,0,0,0,1,1,0,0,0,0,1,0,0
12351.0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18283.0,0,0,1,2,0,0,0,0,1,0,0,2,0
18284.0,0,0,0,0,0,0,0,0,0,0,2,0,0
18285.0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [64]:
# Store November 2011 sales data column name as a list
target = ['2010-12-01 ']

# Store target value as `Y`
Y = cust_month_tx[target]

In [68]:
# Store customer identifier column name as a list
custid = ['Customer ID']

# Select feature column names excluding customer identifier
cols = [col for col in features.columns if col not in custid]

# Extract the features as `X`
X = features[cols]

# Split data to training and testing
train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.25, random_state=99)

### Predict next month transactions

In [70]:
# Initialize linear regression instance
linreg = LinearRegression()

# Fit the model to training dataset
linreg.fit(train_X, train_Y)

# Predict the target variable for training data
train_pred_Y = linreg.predict(train_X)

# Predict the target variable for testing data
test_pred_Y = linreg.predict(test_X)

In [72]:
# Calculate root mean squared error on training data
rmse_train = np.sqrt(mean_squared_error(train_Y, train_pred_Y))

# Calculate mean absolute error on training data
mae_train = mean_absolute_error(train_Y, train_pred_Y)

# Calculate root mean squared error on testing data
rmse_test = np.sqrt(mean_squared_error(test_Y, test_pred_Y))

# Calculate mean absolute error on testing data
mae_test = mean_absolute_error(test_Y, test_pred_Y)

# Print the performance metrics
print('RMSE train: {}; RMSE test: {}\nMAE train: {}, MAE test: {}'.format(rmse_train, rmse_test, mae_train, mae_test))

RMSE train: 0.6932454929335424; RMSE test: 0.652239694851849
MAE train: 0.2955689430604907, MAE test: 0.29801541632971634
